# ETL Process for CSV Files
This notebook demonstrates an ETL process for the CSV files in the directory.

In [35]:
# Import Required Libraries
import pandas as pd
import numpy as np
import re

In [36]:
DEFAULT_EMAIL = 'na@test.com'
DEFAULT_PHONE = '000-000-0000'
DEFAULT_DOB = '01/01/1900'
DEFAULT_STREET = '123 Main St'
DEFAULT_CITY = 'Anytown'
DEFAULT_STATE = 'NY'
DEFAULT_ZIP = '20000'
DEFAULT_SEX = 'Other'


## Load CSV Files
Load the CSV files from the directory into pandas DataFrames.

In [59]:
# Load CSV files
athletes_df = pd.read_csv('athletes.csv', encoding='latin1')

# Display the first rows of the DataFrame
print(athletes_df.columns.tolist())
print(athletes_df.head())

['AthleteId', 'Parent2Email', 'GrpHome', 'Active', 'LastName', 'FirstName', 'Sex', 'DateOfBirth', 'SSN', 'ParentName', 'ParentAddress', 'ParentCity', 'ParentState', 'ParentZIP', 'ParentPhone', 'ParentCell', 'ParentEMail', 'Parent2Name', 'Parent2Address', 'Parent2City', 'Parent2State', 'Parent2Zip', 'Parent2Phone', 'Parent2Cell', 'HomePhone', 'EmergencyNbr', 'AthleteAddress', 'AthleteCity', 'AthleteState', 'AthleteZip', 'AthleteEMail', 'CounselorName', 'CounselorEMail', 'CounselorPhone', 'AgencyName', 'DrNamePhone', 'Downs', 'AaAx', 'NoXray', 'Photo', 'Na/Rl', 'Allgeries', 'Tetnus', 'Medication', 'Comments', 'MedicalDeadline', 'PhotoRelease', 'AlternatePhone', 'EntryDate', 'TorchRunner', 'AwardsComments', 'Deactivation_Dt']
   AthleteId            Parent2Email GrpHome  Active              LastName  \
0       1445     carterg22@gmail.com      NO    True                   NaN   
1       1320                     NaN      NO   False                   NaN   
2       1477                     

## Normalization and Cleaner functions here

In [38]:
# Define a function to normalize phone numbers
def normalize_phone(phone):
    if pd.notna(phone):
        phone = str(phone).strip()
        # Remove non-numeric characters
        phone = re.sub(r'\D', '', phone)
        # Format to standard 10-digit format
        if len(phone) == 10:
            return f"{phone[:3]}-{phone[3:6]}-{phone[6:]}"
    return DEFAULT_PHONE

# Define a function to extract phone numbers from a string
def extract_phone(s):
    if pd.isna(s):
        return None, None
    s = str(s)
    # Regex to find phone numbers in various formats
    phone_match = re.search(r'(\d{3}[-.]?\d{3}[-.]?\d{4}\b)', s)
    if phone_match:
        phone = phone_match.group(1)
        # Remove the phone number from the string to get the name
        name = s.replace(phone, '').strip(', /-')
        return name, phone
    return s, None

# Define a function to normalize email
def normalize_email(email):
    if pd.notna(email) and '@' in str(email):
        return str(email).strip().lower()
    return DEFAULT_EMAIL

# Define a function to split full names into first and last names
def split_name(full_name):
    if pd.notna(full_name) and full_name.strip():
        parts = full_name.strip().split()
        if len(parts) > 1:
            return parts[0], ' '.join(parts[1:])
        return parts[0], ''
    return 'N/A', 'N/A'

def counselor_exists(row):
    first_name = row.get('CounselorfirstName')
    last_name = row.get('CounselorlastName')
    return pd.notna(first_name) and first_name not in ['', 'N/A'] and pd.notna(last_name) and last_name not in ['', 'N/A']

# Export Parents/Guardians Data
Create a separate CSV file for parents/guardians, linking them to athletes using AthleteId.

In [61]:
# Define a function to map parent/guardian data based on a prefix
def map_parent(row, prefix):
    if prefix == "Parent2":
        email_key = "Parent2Email"
        zip_col = f"{prefix}Zip"
    else:
        email_key = f"{prefix}EMail"
        zip_col = f"{prefix}ZIP"

    first_name, last_name = split_name(row[f"{prefix}Name"])

    return {
        "id": row["AthleteId"],
        "firstName": first_name,
        "lastName": last_name,
        "primaryPhone": normalize_phone(row[f"{prefix}Phone"]),
        "secondaryPhone": (lambda p: p if p != DEFAULT_PHONE else "")(
            normalize_phone(row.get(f"{prefix}Cell"))
        ),
        "email": normalize_email(row.get(email_key)),
        "street": row.get(f"{prefix}Address"),
        "city": row.get(f"{prefix}City"),
        "state": row.get(f"{prefix}State"),
        "zip": row.get(zip_col)
    }


mapped_parents = []
# Iterate over each row in the athletes_df DataFrame
for _, row in athletes_df.iterrows():
    # Map Parent 1 if required fields are present
    if pd.notna(row["ParentName"]) and pd.notna(row["ParentPhone"]):
        mapped_parents.append(map_parent(row, "Parent"))
    # Map Parent 2 if required fields are present
    if pd.notna(row["Parent2Name"]) and pd.notna(row["Parent2Phone"]):
        mapped_parents.append(map_parent(row, "Parent2"))

# Convert the mapped data to a DataFrame and export to CSV
mapped_parents_df = pd.DataFrame(mapped_parents)

# Filter out rows missing first or last name
mask_missing = (
    mapped_parents_df['firstName'].isna() |
    mapped_parents_df['lastName'].isna() |
    (mapped_parents_df['firstName'] == '') |
    (mapped_parents_df['lastName'] == '') |
    (mapped_parents_df['firstName'] == 'N/A') |
    (mapped_parents_df['lastName'] == 'N/A')
)
if mask_missing.any():
    for _, row in mapped_parents_df[mask_missing].iterrows():
        print(
            f"Dropping parent record id={row['id']} "
            f"due to missing name: firstName={row['firstName']!r}, "
            f"lastName={row['lastName']!r}"
        )
    mapped_parents_df = mapped_parents_df.loc[~mask_missing].copy()

# Save to CSV and print confirmation
mapped_parents_df.to_csv('athlete_parents.csv', index=False)
print('Mapped parent data saved to athlete_parents.csv')

Dropping parent record id=224 due to missing name: firstName='465-6212', lastName=''
Dropping parent record id=601 due to missing name: firstName='ARC', lastName=''
Dropping parent record id=124 due to missing name: firstName='Donna', lastName=''
Mapped parent data saved to athlete_parents.csv


In [62]:
# Strip parent-related columns from athletes_df now that they are no longer needed
columns_to_drop = ['ParentName', 'ParentPhone', 'ParentCell', 'ParentEMail', 'ParentAddress', 'ParentCity', 'ParentState', 'ParentZIP',
                  'Parent2Name', 'Parent2Phone', 'Parent2Cell', 'Parent2Email', 'Parent2Address', 'Parent2City', 'Parent2State', 'Parent2Zip']

cols_lower = {col.lower(): col for col in athletes_df.columns}
existing_columns = [cols_lower[c.lower()] for c in ['ParentName', 'ParentPhone', 'ParentCell', 'ParentEMail', 'ParentAddress', 'ParentCity', 'ParentState', 'ParentZIP',
                                                     'Parent2Name', 'Parent2Phone', 'Parent2Cell', 'Parent2Email', 'Parent2Address', 'Parent2City', 'Parent2State', 'Parent2Zip']
                    if c.lower() in cols_lower]
if existing_columns:
    athletes_df.drop(existing_columns, axis=1, inplace=True)
    print("Parent-related columns removed.")
else:
    print("No parent-related columns to remove.")

Parent-related columns removed.


In [63]:
# print just the headers of the DataFrame
print(athletes_df.columns.tolist())

['AthleteId', 'GrpHome', 'Active', 'LastName', 'FirstName', 'Sex', 'DateOfBirth', 'SSN', 'HomePhone', 'EmergencyNbr', 'AthleteAddress', 'AthleteCity', 'AthleteState', 'AthleteZip', 'AthleteEMail', 'CounselorName', 'CounselorEMail', 'CounselorPhone', 'AgencyName', 'DrNamePhone', 'Downs', 'AaAx', 'NoXray', 'Photo', 'Na/Rl', 'Allgeries', 'Tetnus', 'Medication', 'Comments', 'MedicalDeadline', 'PhotoRelease', 'AlternatePhone', 'EntryDate', 'TorchRunner', 'AwardsComments', 'Deactivation_Dt']


## Export Counselor Data
Create a separate CSV file for counselors, linking them to athletes using AthleteId.

In [42]:
# Define an improved function to split agency name and phon
def split_agency_name(agency):
    if pd.notna(agency):
        # Try different regex patterns to extract phone numbers
        # Pattern 1: Name, phone format
        match = re.search(r'(.*?)[, /\s]+(\d{3}[-\s]?\d{3}[-\s]?\d{4})$', agency.strip())
        if match:
            return match.group(1).strip(), match.group(2).strip()
        
        # Pattern 2: Name/phone format
        match = re.search(r'(.*?)[/](\d{3}[-\s]?\d{3}[-\s]?\d{4})$', agency.strip())
        if match:
            return match.group(1).strip(), match.group(2).strip()
            
        # Pattern 3: Name (phone) format
        match = re.search(r'(.*?)\s*\(\s*(\d{3}[-\s]?\d{3}[-\s]?\d{4})\s*\)', agency.strip())
        if match:
            return match.group(1).strip(), match.group(2).strip()
        
        # No phone number found, return just the name
        return agency.strip(), None
    return None, None

# Define a function to normalize agency names (remove trailing commas, etc.)
def normalize_agency_name(name):
    if pd.notna(name):
        # Remove trailing commas, slashes, etc.
        return re.sub(r'[,/]$', '', name.strip())
    return None

# Function to map common agency names to a standard form
def standardize_agency_name(name):
    if pd.notna(name):
        # Map of common variations to standard names
        agency_map = {
            'Linwood': 'Linwood Center',
            'Linwood,': 'Linwood Center',
            'ARC': 'Arc of Howard County',
            'ARC/': 'Arc of Howard County',
            'ARC /': 'Arc of Howard County',
            'Athelas': 'Athelas Institute',
            'Athelas,': 'Athelas Institute',
            'Athlelas,': 'Athelas Institute',
            'Athelas Institute,': 'Athelas Institute'
        }
        
        # Check if the name is a known variation
        clean_name = normalize_agency_name(name)
        if clean_name in agency_map:
            return agency_map[clean_name]
        return clean_name
    return None

In [65]:
#Check for missing columns
print("Available columns in DataFrame:", athletes_df.columns.tolist())

Available columns in DataFrame: ['AthleteId', 'GrpHome', 'Active', 'LastName', 'FirstName', 'Sex', 'DateOfBirth', 'SSN', 'HomePhone', 'EmergencyNbr', 'AthleteAddress', 'AthleteCity', 'AthleteState', 'AthleteZip', 'AthleteEMail', 'CounselorName', 'CounselorEMail', 'CounselorPhone', 'AgencyName', 'DrNamePhone', 'Downs', 'AaAx', 'NoXray', 'Photo', 'Na/Rl', 'Allgeries', 'Tetnus', 'Medication', 'Comments', 'MedicalDeadline', 'PhotoRelease', 'AlternatePhone', 'EntryDate', 'TorchRunner', 'AwardsComments', 'Deactivation_Dt']


In [66]:
# Create a copy for counselor processing
counselors_df = athletes_df[['AthleteId', 'CounselorName', 'CounselorEMail', 'CounselorPhone', 'AgencyName']].copy().dropna(subset=['CounselorName'])
counselors_df.rename(columns={'AthleteId': 'id'}, inplace=True)

# Apply extraction and cleaning functions for counselor name and phone
counselors_df[['CounselorName_clean', 'phone_from_name']] = counselors_df['CounselorName'].apply(lambda x: pd.Series(extract_phone(x)))
counselors_df['phone'] = counselors_df['CounselorPhone'].fillna(counselors_df['phone_from_name']).apply(normalize_phone)
counselors_df['email'] = counselors_df['CounselorEMail'].apply(normalize_email)
counselors_df[['firstName', 'lastName']] = counselors_df['CounselorName_clean'].apply(lambda x: pd.Series(split_name(x)))

# Apply extraction and cleaning functions for agency name and phone
counselors_df[['agencyName', 'agencyPhone']] = counselors_df['AgencyName'].apply(lambda x: pd.Series(split_agency_name(x)))
counselors_df['agencyName'] = counselors_df['agencyName'].apply(standardize_agency_name)

# Create a map of the first phone number for each agency
agency_phone_map = counselors_df.dropna(subset=['agencyName', 'agencyPhone']).groupby('agencyName')['agencyPhone'].first().to_dict()

# Fill missing agency phones using the map
counselors_df['agencyPhone'] = counselors_df['agencyName'].map(agency_phone_map).fillna(counselors_df['agencyPhone'])

# Normalize the agency phone numbers
counselors_df['agencyPhone'] = counselors_df['agencyPhone'].apply(normalize_phone)

# If an agency name exists, a phone must be present; otherwise, slash them
mask = counselors_df['agencyName'].notna() & (counselors_df['agencyPhone'] == DEFAULT_PHONE)
counselors_df.loc[mask, ['agencyName', 'agencyPhone']] = [None, None]

# Filter out rows that are likely not names
counselors_df = counselors_df[~counselors_df['firstName'].str.contains('Adaptive|Living|Center|School|Rehab|Apartment|Arc', na=False, case=False)]

# Prefix the split name and contact columns with “Counselor”
counselors_df.rename(columns={
    'firstName': 'CounselorfirstName',
    'lastName':  'CounselorlastName',
    'email':     'Counseloremail',
    'phone':     'Counselorphone'
}, inplace=True)
# Select and rename final columns
final_counselors_df = counselors_df[['id', 'CounselorfirstName', 'CounselorlastName', 'Counseloremail', 'Counselorphone', 'agencyName', 'agencyPhone']].copy()

# Drop rows where firstName and lastName are both missing/invalid
final_counselors_df.dropna(subset=['CounselorfirstName', 'CounselorlastName'], how='all', inplace=True)
final_counselors_df = final_counselors_df[final_counselors_df['CounselorfirstName'] != '']

# Save the processed counselors data
final_counselors_df.to_csv('athlete_counselors.csv', index=False)
print('Counselor data saved to athlete_counselors.csv')

# Drop counselor-related columns from athletes_df
counselor_columns = ['CounselorName', 'CounselorEMail', 'CounselorPhone', 'AgencyName', 'GrpHome']
existing_counselor_columns = [col for col in counselor_columns if col in athletes_df.columns]
if existing_counselor_columns:
    athletes_df.drop(existing_counselor_columns, axis=1, inplace=True)
    print(f"Dropped counselor columns: {existing_counselor_columns}")
else:
    print("No counselor columns to drop")

Counselor data saved to athlete_counselors.csv
Dropped counselor columns: ['CounselorName', 'CounselorEMail', 'CounselorPhone', 'AgencyName', 'GrpHome']


## Export Athlete Data
Create a separate CSV file for athletes matching the simplified schema.

## Normalize DOB

In [67]:
# Inspect and clean 'DateOfBirth' column
if 'DateOfBirth' in athletes_df.columns:
    # Replace invalid or missing string values with pd.NA
    athletes_df['DateOfBirth'] = athletes_df['DateOfBirth'].replace(['', 'None', 'N/A', 'NaN'], pd.NA)

    # Convert to datetime, coercing errors to NaT (Not a Time)
    athletes_df['DateOfBirth'] = pd.to_datetime(athletes_df['DateOfBirth'], errors='coerce', format='%d-%b-%y')

    # Define a function to adjust years that are incorrectly parsed into the future
    def fix_year(date):
        if pd.notna(date) and date.year > 2025:  # Assuming no valid DOB is in the future
            return date.replace(year=date.year - 100)
        return date

    # Apply the year correction
    athletes_df['DateOfBirth'] = athletes_df['DateOfBirth'].apply(fix_year)

    # Fill any remaining NaT values with the default DOB
    athletes_df['DateOfBirth'] = athletes_df['DateOfBirth'].fillna(pd.to_datetime(DEFAULT_DOB, format='%m/%d/%Y'))

    # Format the final date strings to MM/DD/YYYY
    athletes_df['DateOfBirth'] = athletes_df['DateOfBirth'].dt.strftime('%m/%d/%Y')

    print("'DateOfBirth' column cleaned and formatted.")
    print(athletes_df[['DateOfBirth']].head())
else:
    print("Error: 'DateOfBirth' column is missing in the DataFrame.")

'DateOfBirth' column cleaned and formatted.
  DateOfBirth
0  01/30/2014
1  01/01/1900
2  08/13/2012
3  09/25/2001
4  11/20/1983


In [68]:
# Ensure the required columns exist before extracting athlete data
required_columns = ['AthleteId', 'FirstName', 'LastName', 'Sex', 'DateOfBirth', 'AthleteEMail', 'HomePhone', 'EmergencyNbr',
                'AthleteAddress', 'AthleteCity', 'AthleteState', 'AthleteZip', 'Active', 'MedicalDeadline']


# Extract athlete data matching the simplified schema
athletes_schema_df = athletes_df[required_columns].copy()

# Add uniform_size column with default value 's'
athletes_schema_df['uniform_size'] = 's'

# Define a function to normalize phone numbers (extract only digits)
def normalize_phone_digits(phone):
    if pd.isna(phone) or phone == '':
        return ''
    # Extract only digits from the phone number
    return ''.join(filter(str.isdigit, str(phone)))

# Normalize phone numbers to extract digits first
athletes_schema_df['HomePhone'] = athletes_schema_df['HomePhone'].apply(normalize_phone_digits)
athletes_schema_df['EmergencyNbr'] = athletes_schema_df['EmergencyNbr'].apply(normalize_phone_digits)

# If HomePhone is missing, use EmergencyNbr
athletes_schema_df['HomePhone'] = athletes_schema_df['HomePhone'].replace('', np.nan).fillna(athletes_schema_df['EmergencyNbr'])

# Now, apply the final normalization that defaults to DEFAULT_PHONE
athletes_schema_df['HomePhone'] = athletes_schema_df['HomePhone'].apply(normalize_phone)

# Validate and normalize emails
if 'AthleteEMail' in athletes_schema_df.columns:
    athletes_schema_df['AthleteEMail'] = athletes_schema_df['AthleteEMail'].apply(normalize_email)

# Fill missing address parts with default values
for col, default in [('AthleteAddress', DEFAULT_STREET),
                     ('AthleteCity', DEFAULT_CITY),
                     ('AthleteState', DEFAULT_STATE),
                     ('AthleteZip', DEFAULT_ZIP)]:
    athletes_schema_df[col] = athletes_schema_df[col].replace('', np.nan).fillna(default)

# Add county for Maryland athletes, defaulting to "howard county"
athletes_schema_df['county'] = np.where(athletes_schema_df['AthleteState'] == 'MD', 'howard county', '')

# default sex to Default value if not present
if 'Sex' in athletes_schema_df.columns:
    athletes_schema_df['Sex'] = athletes_schema_df['Sex'].replace('', np.nan).fillna(DEFAULT_SEX)

athletes_schema_df.drop(columns=['EmergencyNbr'], inplace=True)

# Rename columns to match the schema
athletes_schema_df.rename(columns={
    'AthleteId': 'id',
    'FirstName': 'firstName',
    'LastName': 'lastName',
    'Sex': 'gender',
    'DateOfBirth': 'dateOfBirth',
    'AthleteEMail': 'email',
    'HomePhone': 'phone', 
    'AthleteAddress': 'street',
    'AthleteCity': 'city',
    'AthleteState': 'state',
    'AthleteZip': 'zip',
    'Active': 'status',
    'MedicalDeadline': 'medicalStatus'
}, inplace=True)

# Convert 'status' to a more descriptive format
athletes_schema_df['status'] = athletes_schema_df['status'].apply(lambda x: 'Active' if x else 'Inactive')

# Format medical status dates from "28-Mar-21" to "03/04/2023" format
def format_medical_date(date_str):
    if pd.isna(date_str) or date_str == '':
        return ''
    try:
        # Parse the date string
        date_obj = pd.to_datetime(date_str, format='%d-%b-%y', errors='coerce')
        if pd.isna(date_obj):
            return date_str
        
        # Format as MM/DD/YYYY
        return date_obj.strftime('%m/%d/%Y')
    except:
        return date_str

athletes_schema_df['medicalStatus'] = athletes_schema_df['medicalStatus'].apply(format_medical_date)

# Save to a new CSV file
athletes_schema_df.to_csv('athletes_simplified.csv', index=False)
print('Athlete data saved to athletes_simplified.csv')

Athlete data saved to athletes_simplified.csv


## Drop Simplified Athlete Columns
Remove columns that were exported to `athletes_simplified.csv`, excluding `FirstName`, `LastName` and `AthleteId`.

In [69]:
# Define columns to drop (excluding 'FirstName', 'LastName' and 'AthleteId')
columns_to_drop = ['Sex', 'DateOfBirth', 'AthleteEMail', 'HomePhone',
                   'AthleteAddress', 'AthleteCity', 'AthleteState', 'AthleteZip', 'Active', 'MedicalDeadline', 'EmergencyNbr']


# Drop columns if they exist in the DataFrame
existing_columns = [col for col in columns_to_drop if col in athletes_df.columns]
if existing_columns:
    athletes_df.drop(existing_columns, axis=1, inplace=True)
    print(f"Dropped columns: {existing_columns}")
else:
    print("No matching columns to drop.")

# Print the remaining headers of the DataFrame
print('Remaining headers:', athletes_df.columns.tolist())

Dropped columns: ['Sex', 'DateOfBirth', 'AthleteEMail', 'HomePhone', 'AthleteAddress', 'AthleteCity', 'AthleteState', 'AthleteZip', 'Active', 'MedicalDeadline', 'EmergencyNbr']
Remaining headers: ['AthleteId', 'LastName', 'FirstName', 'SSN', 'DrNamePhone', 'Downs', 'AaAx', 'NoXray', 'Photo', 'Na/Rl', 'Allgeries', 'Tetnus', 'Medication', 'Comments', 'PhotoRelease', 'AlternatePhone', 'EntryDate', 'TorchRunner', 'AwardsComments', 'Deactivation_Dt']


In [70]:
# Check for comments columns and export them if they exist
if 'Comments' in athletes_df.columns and 'AwardsComments' in athletes_df.columns:
    comments_df = athletes_df[['AthleteId', 'Comments', 'AwardsComments']].copy()
    comments_df.rename(columns={
        'Comments': 'normal_comments',
        'AwardsComments': 'award_comments'
    }, inplace=True)
    comments_df.to_csv('athlete_comments.csv', index=False)
    print('Athlete comments CSV exported as athlete_comments.csv')
    athletes_df.drop(['Comments', 'AwardsComments'], axis=1, inplace=True)
    print('Dropped comments columns from athletes_df')
else:
    print('No comment columns found to export or drop')

print('Remaining headers:', athletes_df.columns.tolist())

Athlete comments CSV exported as athlete_comments.csv
Dropped comments columns from athletes_df
Remaining headers: ['AthleteId', 'LastName', 'FirstName', 'SSN', 'DrNamePhone', 'Downs', 'AaAx', 'NoXray', 'Photo', 'Na/Rl', 'Allgeries', 'Tetnus', 'Medication', 'PhotoRelease', 'AlternatePhone', 'EntryDate', 'TorchRunner', 'Deactivation_Dt']


## Extract Medical Data
Create a separate DataFrame for medical information including doctor details and allergies/medication.

In [71]:
# Define a function to split doctor name and phone number
def split_doctor_info(doctor_info):
    if pd.isna(doctor_info) or doctor_info == '':
        return None, None
    
    # Common pattern: "Dr. Name, Phone Number"
    match = re.search(r'(.*?)(?:,\s*(\d{3}[-\s]?\d{3}[-\s]?\d{4}))?$', doctor_info.strip())
    if match:
        name = match.group(1).strip() if match.group(1) else None
        phone = match.group(2).strip() if match.group(2) else None
        return name, phone
    else:
        return doctor_info, None

# Create medical_df from athletes_df with required columns
medical_columns = ['AthleteId', 'DrNamePhone', 'Allgeries', 'Medication']
medical_df = athletes_df[medical_columns].copy()

# Rename AthleteId to id
medical_df.rename(columns={'AthleteId': 'id'}, inplace=True)

# Split DrNamePhone into doctor_name and doctor_phone
medical_df[['doctor_name', 'doctor_phone']] = medical_df['DrNamePhone'].apply(lambda x: pd.Series(split_doctor_info(x)))

# If doctor_name is missing, slash both name and phone
mask = medical_df['doctor_name'].isna() | (medical_df['doctor_name'].str.strip() == '')
medical_df.loc[mask, ['doctor_name', 'doctor_phone']] = [None, None]

# Normalize the doctor_phone column and ensure it's a string
medical_df['doctor_phone'] = medical_df['doctor_phone'].apply(lambda x: str(normalize_phone(x)) if x else '')

# Drop the original DrNamePhone column
medical_df.drop(['DrNamePhone'], axis=1, inplace=True)

# Reorder columns to match the desired format
medical_df = medical_df[['id', 'doctor_name', 'doctor_phone', 'Allgeries', 'Medication']]

# Save the medical data to a CSV file
medical_df.to_csv('athlete_medical.csv', index=False)
print('Medical data saved to athlete_medical.csv')

# Display a sample of the medical data
print("\nSample of medical data:")
print(medical_df.head())

# Drop the medical columns from the athletes_df since they are now in their own file
medical_columns_to_drop = ['DrNamePhone', 'Allgeries', 'Medication', 'Downs', 'AaAx', 'NoXray', 'Tetnus']
existing_medical_columns = [col for col in medical_columns_to_drop if col in athletes_df.columns]
if existing_medical_columns:
    athletes_df.drop(existing_medical_columns, axis=1, inplace=True)
    print(f"\nDropped medical columns: {existing_medical_columns}")
else:
    print("\nNo medical columns to drop")

# Print remaining columns
print("\nRemaining headers:", athletes_df.columns.tolist())

Medical data saved to athlete_medical.csv

Sample of medical data:
     id         doctor_name  doctor_phone          Allgeries  \
0  1445  Dr. Michelle Mcwan  443-451-1600                NaN   
1  1320                None                              NaN   
2  1477       Dr. L. Berger  410-465-7550                NaN   
3  1034         Dr. Hashimi  410-997-2770                NaN   
4    18       Dr. Alice Lee                augmentin, gluten   

                                          Medication  
0  Zoloft, 75mg, qd\nClanidine, 2mg, qhs\nExlax, ...  
1                                                NaN  
2                                                NaN  
3  insulin aspert, 3xday\ndexem G7, every 10days\...  
4  clonazapam, 0.5mg, 2xday\nLevothyroxine, 50mcg...  

Dropped medical columns: ['DrNamePhone', 'Allgeries', 'Medication', 'Downs', 'AaAx', 'NoXray', 'Tetnus']

Remaining headers: ['AthleteId', 'LastName', 'FirstName', 'SSN', 'Photo', 'Na/Rl', 'PhotoRelease', 'AlternatePh

# Generate Nested JSON from CSVs
This section generates a nested JSON file from the exported CSV files.

In [72]:
# First, let's create the initial nested JSON file from our CSV files
import json
import pandas as pd
import numpy as np


# Load the simplified athletes data
athletes_df = pd.read_csv('athletes_simplified.csv', encoding='latin1')
print(f"Loaded {len(athletes_df)} athletes from simplified CSV")

# Load the medical data
medical_df = pd.read_csv('athlete_medical.csv', encoding='latin1')
print(f"Loaded {len(medical_df)} medical records")

# Load the counselors data
counselors_df = pd.read_csv('athlete_counselors.csv', encoding='latin1')
print(f"Loaded {len(counselors_df)} counselor records")

# Load the comments data
comments_df = pd.read_csv('athlete_comments.csv', encoding='latin1')
print(f"Loaded {len(comments_df)} comment records")

# Load the parents data
parents_df = pd.read_csv('athlete_parents.csv', encoding='latin1')
print(f"Loaded {len(parents_df)} parent records")


# Custom JSON serializer to handle different types
def custom_json_serializer(obj):
    if isinstance(obj, (np.integer)):
        return int(obj)
    elif isinstance(obj, (np.floating)):
        return str(int(obj)) if obj.is_integer() else str(obj)
    elif isinstance(obj, np.ndarray):
        return obj.tolist()
    elif pd.isna(obj):
        return ""
    return obj

# Now create a nested JSON structure
nested_athletes = []

# Process each athlete
if not athletes_df.empty:
    for _, athlete in athletes_df.iterrows():
        athlete_id = athlete['id']
        athlete_dict = athlete.to_dict()
        
        # Add medical information if available
        if not medical_df.empty and 'id' in medical_df.columns:
            medical_records = medical_df[medical_df['id'] == athlete_id]
            if not medical_records.empty:
                medical_info = medical_records.iloc[0].to_dict()
                # Remove id to avoid duplication
                if 'id' in medical_info:
                    del medical_info['id']
                athlete_dict['medical'] = medical_info
        
        # Add counselor information if available
        if not counselors_df.empty and 'id' in counselors_df.columns:
            counselor_records = counselors_df[counselors_df['id'] == athlete_id]
            if not counselor_records.empty:
                counselor_info = counselor_records.iloc[0].to_dict()
                # Remove id to avoid duplication
                if 'id' in counselor_info:
                    del counselor_info['id']
                athlete_dict['counselor'] = counselor_info
        
        # Add comments if available
        if not comments_df.empty:
            comment_records = comments_df[comments_df['AthleteId'] == athlete_id]
            if not comment_records.empty:
                comment_info = comment_records.iloc[0].to_dict()

                # Check if comments are actually present and not just empty strings
                has_normal_comment = pd.notna(comment_info.get('normal_comments')) and str(comment_info.get('normal_comments')).strip()
                has_award_comment = pd.notna(comment_info.get('award_comments')) and str(comment_info.get('award_comments')).strip()

                if has_normal_comment or has_award_comment:
                    # Remove id to avoid duplication
                    if 'AthleteId' in comment_info:
                        del comment_info['AthleteId']
                    athlete_dict['comments'] = comment_info
        
        # Add parents as an array if available
        if not parents_df.empty:
            parent_records = parents_df[parents_df['id'] == athlete_id]
            if not parent_records.empty:
                parents_list = []
                for _, parent in parent_records.iterrows():
                    parent_info = parent.to_dict()
                    # Remove athlete id to avoid duplication
                    if 'id' in parent_info:
                        del parent_info['id']
                    parents_list.append(parent_info)
                athlete_dict['parents'] = parents_list
        
        nested_athletes.append(athlete_dict)

# Write the nested data to a JSON file
json_file_path = 'athletes_nested.json'
with open(json_file_path, 'w') as f:
    json.dump(nested_athletes, f, indent=4, default=custom_json_serializer)

print(f"Created nested JSON file with {len(nested_athletes)} athletes")

# Display a sample of the nested data if available
if nested_athletes:
    print("\nSample of first athlete in nested JSON:")
    print(json.dumps(nested_athletes[0], indent=4, default=custom_json_serializer))
else:
    print("No athletes to include in nested JSON")

Loaded 1150 athletes from simplified CSV
Loaded 1150 medical records
Loaded 104 counselor records
Loaded 104 counselor records
Loaded 1150 comment records
Loaded 769 parent records
Loaded 1150 comment records
Loaded 769 parent records
Created nested JSON file with 1150 athletes

Sample of first athlete in nested JSON:
{
    "id": 1445,
    "firstName": NaN,
    "lastName": NaN,
    "gender": "F",
    "dateOfBirth": "01/30/2014",
    "email": "na@test.com",
    "phone": "000-000-0000",
    "street": "7036 Foxton Way",
    "city": "Hanover",
    "state": "MD",
    "zip": "21076",
    "status": "Active",
    "medicalStatus": "03/05/2027",
    "uniform_size": "s",
    "county": "howard county",
    "medical": {
        "doctor_name": "Dr. Michelle Mcwan",
        "doctor_phone": "443-451-1600",
        "Allgeries": NaN,
        "Medication": "Zoloft, 75mg, qd\nClanidine, 2mg, qhs\nExlax, 1tablet, qd\nMiralas, 1 cap, qd"
    },
    "comments": {
        "normal_comments": "Autism",
        

In [73]:
import json
import math
import re

json_file_path = 'athletes_nested.json'

# Step 1: Read the file content as a string
try:
    with open(json_file_path, 'r') as f:
        file_content = f.read()
except FileNotFoundError:
    print(f"Error: File {json_file_path} not found.")
    # Create an empty list if file not found to avoid further errors, or handle appropriately
    file_content = '[]'

# Step 2: Replace standalone NaN with null to make it valid JSON
# Using regex to replace whole word NaN only, avoiding accidental replacement in strings like 'Nancy'
valid_json_content = re.sub(r'\bNaN\b', 'null', file_content)

# Step 3: Parse the modified string
try:
    all_athletes_data = json.loads(valid_json_content)
except json.JSONDecodeError as e:
    print(f"Error decoding JSON: {e}")
    all_athletes_data = []  # Default to empty list on error

original_athlete_count = len(all_athletes_data)

# Step 4 & 5: Define a recursive function to replace None (originally NaN/null) with ''
def replace_none_deep(item):
    if item is None:
        return ''
    elif isinstance(item, dict):
        return {k: replace_none_deep(v) for k, v in item.items()}
    elif isinstance(item, list):
        return [replace_none_deep(elem) for elem in item]
    elif isinstance(item, float) and math.isnan(item):
        return ''
    return item

processed_athletes = []
for athlete in all_athletes_data:
    if not isinstance(athlete, dict):  # Skip if an entry is not a dictionary
        print(f"Skipping non-dictionary entry: {athlete}")
        continue

    # Check for missing name
    first_name = athlete.get('firstName')
    last_name = athlete.get('lastName')

    # A field is considered 'missing' for filtering if it's None (originally NaN/null)
    is_name_missing = not first_name or not last_name

    if is_name_missing:
        continue  # Skip this athlete
    else:
        # Replace all None values (originally NaN/null) with empty strings
        cleaned_athlete = replace_none_deep(athlete)
        processed_athletes.append(cleaned_athlete)

final_athlete_count = len(processed_athletes)

# Step 6: Write the processed data back to the JSON file
with open(json_file_path, 'w') as f:
    json.dump(processed_athletes, f, indent=4)

print(f"Processed {original_athlete_count} athletes. Final count: {final_athlete_count}.")
if processed_athletes:
    print("Sample processed athlete:")
    print(json.dumps(processed_athletes[0], indent=4))
else:
    print("No valid athletes found.")

Processed 1150 athletes. Final count: 1147.
Sample processed athlete:
{
    "id": 1034,
    "firstName": "Mona",
    "lastName": "Abdelhalim",
    "gender": "F",
    "dateOfBirth": "09/25/2001",
    "email": "na@test.com",
    "phone": "410-282-4201",
    "street": "8338 Goverenor Grayson Way",
    "city": "Ellicott City",
    "state": "MD",
    "zip": "21043",
    "status": "Active",
    "medicalStatus": "08/14/2027",
    "uniform_size": "s",
    "county": "howard county",
    "medical": {
        "doctor_name": "Dr. Hashimi",
        "doctor_phone": "410-997-2770",
        "Allgeries": "",
        "Medication": "insulin aspert, 3xday\ndexem G7, every 10days\nLevomyroxine, 175mg, 1xday\nmelatonin, 1xday\nfiber, 1xday\nmultivitamin, 1xday\nsuper green, black see, curcumin, cinamin"
    },
    "parents": [
        {
            "firstName": "Fatima",
            "lastName": "H Kahil",
            "primaryPhone": "410-282-4201",
            "secondaryPhone": "410-831-7156",
            "